In [1]:
import os
import pandas as pd
import numpy as np
from pycaret.regression import *
#from multiprocessing import Pool
from datetime import datetime
import warnings 
warnings.filterwarnings('ignore')
import traceback
#from os.path import exist

pd.set_option('display.min_rows', 21)
pd.set_option('display.max_colwidth', None)

In [2]:
root="/home/paperspace/amex-default-prediction/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
scols = [col for col in headers if col[0] == 'S']
dcols = [col for col in headers if col[0] == 'D']
rcols = [col for col in headers if col[0] == 'R']
bcols = [col for col in headers if col[0] == 'B']
pcols = [col for col in headers if col[0] == 'P']
labels =pd.read_csv(root+"train_labels.csv")

In [4]:
def pdf(df, func, n_cores=6):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
trains = []
for r, d, f in os.walk(root):
    f.sort()
    for file in f:
#        if "traina" in file or "trainb" in file:
        if 'pkl' not in file and 'csv' not in file and ("xa" in file or "xb" in file or file.startswith("xc")):
            trains.append(os.path.join(root, file))

dates=pd.DataFrame(pd.date_range('2018-04-01','2019-04-30', freq='MS').tolist(), columns=['S_2_dt'])
dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

In [18]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

traindf = pd.read_csv(root+"/tdata")
tdf = traindf.groupby('customer_ID')['S_2'].agg(['min', 'max', 'count']).reset_index()
#tdf = tdf.merge(labels, on="customer_ID", how="inner")
#for i in range(13):
#    print(i, tdf[tdf['S_2']==i].count())


In [64]:
tdf.rename(columns={"min": "mindate", "max": "maxdate"}, inplace=True)

tdf['maxdate'] = pd.to_datetime(tdf['maxdate'])
tdf['mindate'] = pd.to_datetime(tdf['mindate'])

#tdf['diffcount'] = tdf.apply(lambda x: ((x['maxdate'] - x['mindate'])/np.timedelta64(1, 'M')))

tdf['mondiff'] = (tdf.maxdate.dt.strftime("%Y%m").astype(int) - tdf.mindate.dt.strftime("%Y%m").astype(int) - 87)
tdf['mondiff'] = tdf['mondiff'].apply(lambda x: x+88 if x< 0 else x)
#tdf[tdf['count'] == tdf['mondiff']]

In [ ]:
tdf[tdf['customer_ID']=='002b66a636237d16b2344db6c39210bae5f091c9caf8479e6066101b4a197ce0']

In [ ]:
traindfs = []

for k in range(13):
    traindfs.append(pd.DataFrame(columns=headers))
k=0
for i in range(len(trains)):
    print(trains[i])
    traindf = pd.read_csv(trains[i], names=headers, header=None)
    for j in range(1, 14):
        mask = traindf.customer_ID.isin(tdf[tdf['mondiff'] == j].customer_ID)
        traindfs[j-1] = pd.concat([traindfs[j-1], traindf[mask]], ignore_index=True)
        if traindfs[j-1]['customer_ID'].count() >= 260000:
#            traindfs[j-1] = traindfs[j-1].merge(labels, on="customer_ID", how="inner")
            traindfs[j-1].iloc[:260000,].to_csv("x" + str(j-1) + "-" + str(k) + ".csv", index=False)
            traindfs[j-1] = traindfs[j-1].iloc[260000:,]
            print("x" + str(j-1) + "-" + str(k) + ".csv")
#            traindfs[j-1].drop(columns=['target'], inplace=True)
            k+=1            

for i in range(len(traindfs)-1):
#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].to_csv("x" + str(i) + ".csv", index=False)
    print("x" + str(i) + ".csv")

for i in range(12, len(traindfs)):
#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].to_csv("x" + str(i) + "-" + str(k) + ".csv", index=False)
    print("x" + str(i) + "-" + str(k) + ".csv")


In [3]:
mdf = pd.DataFrame()
for m in range(1):
    print("x12-" + str(m) + ".csv")
    mdf = pd.concat([mdf, pd.read_csv("x12-"+str(m)+".csv")], ignore_index=True)
mdf['tag'] = 12
mdf['S_2'] = pd.to_datetime(mdf['S_2'])
mdf['S_2'] = mdf['S_2'] + pd.offsets.DateOffset(years=1)
mdf['S_2'] = mdf['S_2'] + pd.offsets.DateOffset(months=1)
mdf['day_of_month'] = mdf['S_2'].dt.strftime("%d")
mdf['day_of_week'] = mdf['S_2'].dt.strftime("%u")
mdf['day_of_year'] = mdf['S_2'].dt.strftime("%j")
mdf['week_of_year'] = mdf['S_2'].dt.isocalendar().week
mdf['yearmonth'] = mdf['S_2'].dt.strftime("%Y%m")

#mdf = mdf.merge(labels, on="customer_ID", how="inner")

x12-0.csv


In [7]:
traindfs = []

print("Loading...", end=" ")
for j in range(12):
    print("x" + str(j), end=" ")
    traindfs.append(pd.read_csv("x" + str(j) + ".csv"))
#    traindfs[j]['tag'] = j
    traindfs[j]['S_2'] = pd.to_datetime(traindfs[j]['S_2'])
    traindfs[j]['day_of_month'] = traindfs[j]['S_2'].dt.strftime("%d")
    traindfs[j]['day_of_week'] = traindfs[j]['S_2'].dt.strftime("%u")
    traindfs[j]['day_of_year'] = traindfs[j]['S_2'].dt.strftime("%j")
    traindfs[j]['week_of_year'] = traindfs[j]['S_2'].dt.isocalendar().week
    traindfs[j]['yearmonth'] = traindfs[j]['S_2'].dt.strftime("%Y%m")

i=0
print("")
print("Transforming...", end=" ")
for j in range(12):
    print("x" + str(j), end=" ")

    maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
    mindate = maxdate + pd.offsets.DateOffset(months= -1-j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=(dates['seq']+1)

    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonthorig'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i]['seq'] = traindfs[i].groupby(['customer_ID']).cumcount().add(1)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = traindfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2018 and x['mondiff'] > j else x['mondiff'], axis=1)
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <= 0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
#    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    traindfs[i]['yearmonth'] = traindfs[i].apply(lambda x: x['yearmonth_y'] if pd.isnull(x['yearmonth_x']) else x['yearmonth['S_2_'], axis=1)
#    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
#    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
#    traindfs[i].drop(columns=['S_2_dt', 'origmissrcnt', 'totmissrcnt', 'seq_x', 'seq_y'], inplace=True)
    traindfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
    traindfs[i] = traindfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    traindfs[i]['yearmonthorig'].fillna(0, inplace=True)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']].fillna(-1, inplace=True)
#    traindfs[i]['mondiff'].fillna(0, inplace=True)
    traindfs[i].drop(columns=['mondiff'], inplace=True)

#    traindfs[i]['tag'] = j
#    traindfs[i] = pd.concat([traindfs[i], pd.merge(mdf, dates['yearmonth'], on=['yearmonth'],how='inner')], ignore_index=True)

#    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
#    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
#    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
#    traindfs[i]['week_of_year'] = traindfs[i]['S_2'].dt.isocalendar().week
#    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i].drop(columns=['S_2'], inplace=True)
##    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")

##    cols=traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
##    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##   traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i][cols] = traindfs[i].groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i].drop(columns=['target'], inplace=True)
##    for k in range(i+1, 12):
##        traindfs[i] = pd.concat([traindfs[i], traindfs[k]], ignore_index=True)

##    traindfs[i].drop(columns=['S_2'], inplace=True)
##    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")

##    cols=traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
##    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i][cols] = traindfs[i].groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i].drop(traindfs[i][traindfs[i]['tag']!=j].index, axis=0, inplace=True)
##    traindfs[i].drop(columns=['tag'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
#    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    traindfs[i].fillna(999, inplace=True)

#    cols=traindfs[i].columns
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in cols if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].groupby("target")[col].transform(lambda x: x.fillna(x.mean()), inplace=True)
#        traindfs[i][col].fillna(-999, inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i].groupby("target")[col].transform(lambda x: x.fillna(x.mean()), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

#    for column in [col for col in traindfs[0].columns if col[len(col)-6:] in ['201904', '201903']]:
#        traindfs[0].drop(columns=[column], inplace=True)

    traindfs[i].to_csv("x" + str(j) +"13dts.csv", index=False)
    i+=1

traindfs[0]

Loading... x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 
Transforming... x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 

,customer_ID,B_1201904,B_10201904,B_11201904,B_12201904,B_13201904,B_14201904,B_15201904,B_16201904,B_17201904,B_18201904,B_19201904,B_2201904,B_20201904,B_21201904,B_22201904,B_23201904,B_24201904,B_25201904,B_26201904,B_27201904,B_28201904,B_29201904,B_3201904,B_30201904,B_31201904,B_32201904,B_33201904,B_36201904,B_37201904,B_38201904,B_39201904,B_4201904,B_40201904,B_41201904,B_42201904,B_5201904,B_6201904,B_7201904,B_8201904,B_9201904,D_102201904,D_103201904,D_104201904,D_105201904,D_106201904,D_107201904,D_108201904,D_109201904,D_110201904,D_111201904,D_112201904,D_113201904,D_114201904,D_115201904,D_116201904,D_117201904,D_118201904,D_119201904,D_120201904,D_121201904,D_122201904,D_123201904,D_124201904,D_125201904,D_126201904,D_127201904,D_128201904,D_129201904,D_130201904,D_131201904,D_132201904,D_133201904,D_134201904,D_135201904,D_136201904,D_137201904,D_138201904,D_139201904,D_140201904,D_141201904,D_142201904,D_143201904,D_144201904,D_145201904,D_39201904,D_41201904,D_42201904,D_43201904,D_44201904,D_45201904,D_46201904,D_47201904,D_48201904,D_49201904,D_50201904,D_51201904,D_52201904,D_53201904,D_54201904,D_55201904,D_56201904,D_58201904,D_59201904,D_60201904,D_61201904,D_62201904,D_63201904,D_64201904,D_65201904,D_66201904,D_68201904,D_69201904,D_70201904,D_71201904,D_72201904,D_73201904,D_74201904,D_75201904,D_76201904,D_77201904,D_78201904,D_79201904,D_80201904,D_81201904,D_82201904,D_83201904,D_84201904,D_86201904,D_87201904,D_88201904,D_89201904,D_91201904,D_92201904,D_93201904,D_94201904,D_96201904,P_2201904,P_3201904,P_4201904,R_1201904,R_10201904,R_11201904,R_12201904,R_13201904,R_14201904,R_15201904,R_16201904,R_17201904,R_18201904,R_19201904,R_2201904,R_20201904,R_21201904,R_22201904,R_23201904,R_24201904,R_25201904,R_26201904,R_27201904,R_28201904,R_3201904,R_4201904,R_5201904,R_6201904,R_7201904,R_8201904,R_9201904,S_11201904,S_12201904,S_13201904,S_15201904,S_16201904,S_17201904,S_18201904,S_19201904,S_20201904,S_22201904,S_23201904,S_24201904,S_25201904,S_26201904,S_27201904,S_3201904,S_5201904,S_6201904,S_7201904,S_8201904,S_9201904,bncnt201904,day_of_month201904,day_of_week201904,day_of_year201904,dncnt201904,origmissrcnt201904,pncnt201904,rncnt201904,sncnt201904,totmissrcnt201904,week_of_year201904,yearmonthorig201904,target
0,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d5ec50ef6aa6487cc14,0.003712,0.028748,0.003663,0.010627,0.006496,0.001786,0.009511,0.002692,NaN,0.585338,0.006257,0.815825,0.006237,0.009585,0.006915,0.211864,0.008420,0.002578,0.001046,0.008517,0.140864,NaN,0.000327,0.0,1,0.008412,1.006920,0.009654,0.005641,1.0,NaN,0.137897,0.094856,0.008053,NaN,0.007317,0.179559,0.228845,1.008720,0.009752,0.810860,1.004064,0.985382,0.512469,NaN,0.336391,NaN,0.004490,NaN,NaN,1.002274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.009784,0.999308,0.005100,0.002419,0.000184,NaN,0.004038,NaN,NaN,NaN,NaN,NaN,0.000577,0.005503,0.001053,NaN,0.003976,0.007178,0.008273,0.007035,0.008014,0.052399,NaN,0.131792,0.011225,NaN,0.555546,0.478355,NaN,NaN,0.008856,0.111624,0.189372,1.009461,0.151432,NaN,0.474101,0.506323,0.001743,0.770476,0.017657,CO,NaN,0.005823,NaN,NaN,NaN,0.001532,0.026944,0.343266,NaN,0.287809,0.276619,0.002294,0.017883,0.002798,0.001638,0.009763,0.006668,NaN,NaN,0.504751,0.009765,NaN,NaN,0.005974,0.003595,0.005987,0.006823,0.001230,0.003505,0.286224,NaN,0.960185,0.507655,0.005379,0.007175,1.004646,0.002560,0.001873,0.000930,0.004070,0.002510,0.001234,0.004018,1.000958,0.008711,0.002895,0.009052,0.002215,0.001069,0.005790,NaN,NaN,0.006742,0.500300,1.005408,0.502572,0.126165,0.143661,1.007309,NaN,0.287190,0.185295,0.004588,0.507485,0.004571,0.007402,0.003437,0.000541,0.009376,0.353586,0.137774,0.158417,0.973855,0.006911,NaN,NaN,0.004802,1.005175,NaN,0.002524,NaN,123,24,6,083,2086,0,3,61,36,0,12,201904,0
1,000eee12732c9c7b790870c556bc33339d6d17725cbeb4a7da6b2a5d2c8386aa,0.079262,0.296491,0.052629,0.013833,0.046850,0.023636,0.003698,0.003267,NaN,1.009979,0.009323,0.816346,0.004377,0.

In [ ]:
traindfs[0] #[(traindfs[3]['origmissrcnt201904'] >0 ) | (traindfs[3]['origmissrcnt201903']>0) | (traindfs[3]['origmissrcnt201902']>0) | (traindfs[3]['origmissrcnt201901']>0)][['origmissrcnt201901', 'origmissrcnt201902', 'origmissrcnt201903', 'origmissrcnt201904']]
#traindfs[0].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
traindfs[2]

In [ ]:
#traindf=pd.read_csv("mdfdtcnt.csv")
traindf=pd.DataFrame()
for j in range(2):
    print(j, end=" ")
    traindf = pd.concat([traindf, traindfs[j]], ignore_index=True)
#    cols=traindf[traindf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    traindf[cols] = traindf.groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindf[cols] = traindf.groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindf[cols] = traindf.groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindf[cols] = traindf.groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindf[cols] = traindf.groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindf[cols] = traindf.groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindf[cols] = traindf.groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

cols=traindf[traindf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

traindf #[['target', 'B_1201804', 'B_10201804', 'B_11201804', 'B_12201804', 'B_13201804', 'B_14201804', 'B_15201804', 'B_16201804', 'B_1201805', 'B_10201805', 'B_11201805', 'B_12201805', 'B_13201805', 'B_14201805', 'B_15201805', 'B_16201805']]

traindf#.groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

In [ ]:
traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))
cols=traindf[traindf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
traindf[['target', 'B_1201806', 'B_10201806', 'B_11201806', 'B_12201806', 'B_13201806', 'B_14201806', 'B_15201806', 'B_16201806', 'B_1201805', 'B_10201805', 'B_11201805', 'B_12201805', 'B_13201805', 'B_14201805', 'B_15201805', 'B_16201805']]


In [ ]:
## grpcols = [col for col in traindfs[i].columns if (col == 'target') or (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]
#grpcols
#traindfs[i].groupby(['yearmonth', 'target'])[col].transform(lambda x: x.fillna(x.mean()))
#yearmonth 	day_of_month 	day_of_week 	day_of_year 	target
#traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[col].agg(['mean'])
#traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[col].agg(['mean'])
#traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[col].agg(['mean'])
#traindfs[i].groupby(['target', 'day_of_week'])[col].agg(['mean'])

cols=mdf[mdf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#mdf[cols] = mdf.groupby(['day_of_year', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))
#mdf[cols] = mdf.groupby(['day_of_year'])[cols].transform(lambda x: x.fillna(x.mean()))
#mdf.groupby(['target', 'day_of_year', 'day_of_week']).transform(lambda x: x.fillna(x.mean()))
traindfs[1]#[['customer_ID', 'B_39201904', 'B_42201904', 'target']]

In [ ]:
traindfs = []
p=0
t=5
print("Loading data parse 1...", end=" ")
for j in range(p):
    print("x12-" + str(j), end=" ")
    traindfs.append(pd.read_csv("x12-" + str(j) + "dtcnt.csv"))

print("Loading data parse 2...", end=" ")
for j in range(p, t):
    print("x12-" + str(j), end=" ")
    traindfs.append(pd.read_csv("x12-" + str(j) + ".csv"))
#    traindfs[j]['tag'] = j
    traindfs[j]['S_2'] = pd.to_datetime(traindfs[j]['S_2'])
    traindfs[j]['day_of_month'] = traindfs[j]['S_2'].dt.strftime("%d")
    traindfs[j]['day_of_week'] = traindfs[j]['S_2'].dt.strftime("%u")
    traindfs[j]['day_of_year'] = traindfs[j]['S_2'].dt.strftime("%j")
    traindfs[j]['week_of_year'] = traindfs[j]['S_2'].dt.isocalendar().week
    traindfs[j]['yearmonth'] = traindfs[j]['S_2'].dt.strftime("%Y%m")

print("")
print("Transforming data...", end=" ")
i=p
j=12
for k in range(p, t):
    print("x" + str(j) + "-" + str(k), end=" ")
    traindfs.append(pd.read_csv("x" + str(j) + "-" + str(k) + ".csv"))

    maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
    mindate = maxdate + pd.offsets.DateOffset(months= -1-j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=(dates['seq']+1)

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonthorig'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i]['seq'] = traindfs[i].groupby(['customer_ID']).cumcount().add(1)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = traindfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2018 and x['mondiff'] > j else x['mondiff'], axis=1)
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <= 0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
#    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    traindfs[i]['yearmonth'] = traindfs[i].apply(lambda x: x['yearmonth_y'] if pd.isnull(x['yearmonth_x']) else x['yearmonth['S_2_'], axis=1)
#    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
#    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
#    traindfs[i] = traindfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    traindfs[i]['yearmonthorig'].fillna(0, inplace=True)
#    traindfs[i]['mondiff'].fillna(0, inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i]['week_of_year'] = traindfs[i]['S_2'].dt.isocalendar().week
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)

#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")

#    cols=traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#    traindfs[i][cols] = traindfs[i].groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

#    traindfs[i].drop(columns=['target'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
#    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    traindfs[i].fillna(0, inplace=True)

#    cols=traindfs[i].columns
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in cols if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(-999, inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

#    for column in [col for col in traindfs[0].columns if col[len(col)-6:] in ['201904', '201903']]:
#        traindfs[0].drop(columns=[column], inplace=True)

    traindfs[i].to_csv("x" + str(j) + "-" + str(k) +"dtcnt.csv", index=False)
    i+=1

traindfs[0]

In [ ]:
traindfs[0].iloc[traindfs[0].notna().index] #[traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index

In [ ]:
traindfs = []

for j in range(len(trains)):
    i = j#-18
    print(trains[j])
    traindfs.append(pd.read_csv(trains[j], names=headers, header=None))

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt'], inplace=True)

    traindfs[i]['mondiff'].fillna(0, inplace=True)
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = traindfs[i].groupby('customer_ID')[col].transform(lambda grp: grp.fillna(np.mean(grp)))
#    for col in traindfs[i].columns[traindfs[i].isna().any()]:
#        if traindfs[i][col].dtype in ['float64', 'int64']:
#            traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in traindfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

    traindfs[i].to_csv(trains[i]+"nona.csv", index=False, header=False)

traindfs[0][traindfs[0]['customer_ID']==0].to_csv(root+"/trains/headers.csv", index=False)
traindfs[0]

In [ ]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

traindf = pd.read_csv(root+"/tdata")


In [ ]:
#traindf = traindf[traindf['target']==1]
#for col in [col for col in traindf.columns if (col[len(col)-6:]=='201804')]:
#    traindf = traindf[~traindf[col].isnull()]

#traindf=traindf[~traindf['B_12201804'].isnull()]
#traindf['B_42201804'].unique()

#(traindf.groupby('customer_ID')['S_2'].agg(["min", "max"]).reset_index())['min'].max(), (traindf.groupby('customer_ID')['S_2'].agg(["min", "max"]).reset_index())['max'].min()

tdf = traindf.groupby('customer_ID')['S_2'].count().reset_index()
tdf = tdf.merge(labels, on="customer_ID", how="inner")
for i in range(1, 13):
    print(i, tdf[tdf['S_2']==i].count())

#traindf.groupby('customer_ID')['S_2'].count().reset_index()

In [ ]:
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns

traindf=pd.DataFrame()

for i in range(12):
    traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new.csv", names=headers, header=None)], ignore_index=True)
print("loaded data...")

for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
    traindf[col] = traindf[col].astype(str)
    traindf[col] = traindf[col].astype('category')
print("updated categories...")

traindf.groupby(['target'])['target'].count()


In [ ]:
s = setup(data=traindf, target='target',
          transformation=True,
          ignore_features=['customer_ID'],
          categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
          numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
#          normalize = True,
          remove_outliers=True, handle_unknown_categorical=True,fix_imbalance=True,
          combine_rare_levels=True, create_clusters=True,
          feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
          train_size=0.8, fold=6,
          use_gpu=True,
          silent = True, session_id = 123, verbose=True)
save_config("/amex/dt3regconfig.pkl")

In [ ]:
%matplotlib inline
#eda()

In [ ]:
#load_config("/amex/dtclassconfig.pkl")
dt = create_model('dt', fold=6, verbose=True)
print("created")
save_model(dt, "/amex/models/dt2reg")

In [ ]:
save_model(dt, "/amex/models/dt1Lclass")

In [ ]:
dt = tune_model(dt, fold=6, n_iter=10, choose_better=True)
print("tuned")
save_model(dt, "e:/rebid/amex/models/dt1Lclasstuned")

In [ ]:
dt = finalize_model(dt)
print("finalized")
#optimize_threshold(dt)
#print("optimized")
save_model(dt, "e:/rebid/amex/models/dt1Lclassfin")

In [ ]:
calibrated_dt = calibrate_model(dt)
save_model(calibrated_dt, "/amex/models/dt1Lclasscalib")

In [ ]:
plot_model(dt, plot='calibration')
plot_model(model, plot='calibration')

In [ ]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=6
algos=['dt', 'catboost', 'lightgbm']
loadconfig=True
loadmodels=[False, False, False]

for i in range(1):
    if loadconfig:
        dtsets.append(load_config(root+"/configs/" + 'dt' + str(n) + "-" + str(i) + "config.pkl"))
    else:
        print(i, end="")
        traindf=pd.DataFrame()
        for j in range(i*n, (i*n)+n):
            print("."+str(j), end="")
            traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new1.csv", names=headers, header=None)], ignore_index=True)

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        dtsets.append(setup(data=traindf, target='target',
              transformation=True,
              ignore_features=['customer_ID'],
              categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
              numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
              remove_outliers=True, handle_unknown_categorical=True,
              combine_rare_levels=True, 
              create_clusters=True,
#              transform_target=True, transform_target_method='yeo-johnson',
              polynomial_features=True,
              feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
              train_size=0.8, fold=6,
              use_gpu=True,
              silent = True, session_id = 123, verbose=True))
        save_config(root+"/configs/" + algo + str(n) + "-" + str(i) + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + str(n)+"-"+str(i)+"reg"))
        else:
            dtmodels.append(create_model(algos[j], fold=6, verbose=True))
            save_model(dtmodels[i], root+"/models/" + algos[j] + str(n)+"-"+str(i)+"reg")

dtmodels.append(stack_models(dtmodels, meta_model = dtmodels[len(dtmodels)-1]))
save_model(dtmodels[len(dtmodels)-1], root+"/models/" + "".join(algos) + "stackedreg")

In [ ]:
# 0 to 12 dates modeling
algos=['catboost']
loadconfig=True
loadmodels=[False, False, False, False]
d=4
nof=4
tag="new"
xfiles = ["x" + str((i*nof)+j) + "13dts.csv" for i in range(int(12/nof)) for j in range(nof)]

mdf=pd.DataFrame()
for m in range(0):
    print("x12-"+str(m)+"dtcnt.csv", end=" ")
    mdf = pd.concat([mdf, pd.read_csv("x12-"+str(m)+"dtcnt.csv")], ignore_index=True)

for i in range(1):
#    xfiles = ["x" + str((i*nof)+j) + "13dts.csv" for j in range(nof)]
    xfiles = ["x" + str(i+j) + "13dts.csv" for j in range(12-i)]
#    xfiles = ["x" + str(i) + "13dts.csv"]
    dtmodels=[]
    tag = "new" + str(i)
    if loadconfig:
        load_config(root+"/configs/" + "x13dts" + tag + "config.pkl")
    else:
        traindf=pd.DataFrame()
        for xfile in xfiles:
            print(xfile, end=" ")
            if xfiles.index(xfile) == 0:
                traindf = pd.concat([traindf, pd.read_csv(xfile)], ignore_index=True)
            else:
#                traindf = pd.concat([traindf, pd.read_csv(xfile)[traindf.columns]], ignore_index=True)
                traindf = pd.concat([traindf, pd.read_csv(xfile)], ignore_index=True)
#            print("train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())

#        if i > 4:
        print("orig train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())
#        traindf = pd.concat([traindf, mdf.loc[0:(i*5000),:][traindf.columns]], ignore_index=True)
#        print("train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or
                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
            transformation=True,
            normalize=True,
            ignore_features=['customer_ID'],
            categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or 
                                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)],
#            numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
            remove_outliers=True,
            handle_unknown_categorical=True,
            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
            trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=2, #d,
            feature_selection=True, feature_selection_threshold=0.9,
#            remove_multicollinearity=True, #pca=True,
            ignore_low_variance=True,
            train_size=0.9, fold=10,
            use_gpu=True,
            silent = True, session_id = 123, verbose=True)
        d -= 1
        save_config(root+"/configs/" + "x13dts" + tag + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x13dts" + tag))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x13dts" + tag)

                if algos[j] in ['dt', 'lightgbm']:
                    try:
                        dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=15, verbose=True)
                        save_model(dtmodels[j], root+"/models/" + algos[j] + "-x13dts" + tag)
                    except:
                        print("Unable to tune for: ", algos[j])
                        traceback.print_exc()
            except:
                print("Unable to build model for:", algos[j])
                traceback.print_exc()

IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:37:49
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


In [ ]:
mdf=pd.read_csv("mdfdtcnt.csv")
pd.concat([mdf, traindf], ignore_index=True)


In [ ]:
dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=15, verbose=True)
save_model(dtmodels[j], root+"/models/" + algos[j] + "-x13dts" + tag)


In [ ]:
algos=['dt', 'catboost', 'lightgbm']
algos=['lightgbm']
loadconfig=True
loadmodels=[False, False, False, False]
d=6

for i in range(12):
    dtmodels=[]
    if loadconfig:
        load_config(root+"/configs/" + "x" + str(i) + "config.pkl")
    else    :
        print("x" + str(i) + "13dts.csv", end="")
        traindf = pd.read_csv("x" + str(i) + "13dts.csv")

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or
                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
            transformation=True,
            normalize=True,
            ignore_features=['customer_ID'],
            categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
            numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
            remove_outliers=True,
            handle_unknown_categorical=True,
            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
            trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=d,
            feature_selection=True, remove_multicollinearity=True, #pca=True,
            ignore_low_variance=True,
            train_size=0.8, fold=10,
            use_gpu=True,
            silent = True, session_id = 123, verbose=True)
        save_config(root+"/configs/" + "x" + str(i) + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x"+str(i)+"reg"))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x"+str(i)+"reg")

                if algos[j] in ['dt', 'lightgbm']:
                    try:
                        dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=30, verbose=True)
                        save_model(dtmodels[j], root+"/models/" + algos[j] + "-x"+str(i)+"reg")
                    except:
                        print("Unable to tune for: ", algos[j])
                        traceback.print_exc()
            except:
                print("Unable to build model for:", algos[j])
                traceback.print_exc()

##    stacked=stack_models(dtmodels, meta_model=dtmodels[len(dtmodels)-1])
##    save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    try:
#        stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
#        save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    except:
#        print("Unable to tune for stacking")
##    dtmodels.append(stacked)
    d -= 1 if ((i/3)==round((i/3))) else d


In [ ]:
algos=['dt', 'catboost', 'lightgbm']
algos=['lightgbm', 'catboost']
loadconfig=[False, False, False, False, False]
loadmodel=[False, False, False, False, False]
tunemodel=[True, True, True, True, True]
d=2
dtmodels=[]
nof=4

for i in range(1):
    tag = "".join([str((i*nof)+j) for j in range(nof)])
    if loadconfig[i]:
        load_config(root+"/configs/" + "x12-" + tag + "config.pkl")
    else:
#        print("x12-" + str(i*nof) + "dtcnt.csv")
#        traindf = pd.read_csv("x12-" + str(i*nof) + "dtcnt.csv")
        traindf=pd.DataFrame()
        for j in range(nof):
#            if exists("x12-" + str((i*nof)+j) + "dtcnt.csv"):
            print("x12-" + str((i*nof)+j) + "dtcnt.csv", end=" ")
            traindf = pd.concat([traindf, pd.read_csv("x12-" + str((i*nof)+j) + "dtcnt.csv")], ignore_index=True)
#            else:
#                print("x12-" + str((i*nof)-1) + "dtcnt.csv")
#                traindf = pd.concat([traindf, pd.read_csv("x12-" + str((i*nof)-1) + "dtcnt.csv")], ignore_index=True)

#        traindf.to_csv("x12-"+str((i*3))+str((i*3)+1)+str((i*3)+2)+".csv")
#        print("wrote x12-"+str((i*3))+str((i*3)+1)+str((i*3)+2)+".csv")
        print("Loaded data...", "train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or
                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]:
#            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
            transformation=True,
            normalize=True,
            ignore_features=['customer_ID'],
            categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)],
#            numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
            remove_outliers=True,
            handle_unknown_categorical=True,
            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
#            trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=d,
            feature_selection=True, feature_selection_threshold=0.9,
#            remove_multicollinearity=True, #pca=True,
            ignore_low_variance=True,
            train_size=0.8, fold=10,
            use_gpu=True,
            silent = True, session_id = 123, verbose=True)
        traindf=pd.DataFrame() #to free up the memory
        save_config(root+"/configs/" + "x12-" + tag + "config.pkl")

    for j in range(len(algos)):
        if loadmodel[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x12-"+tag+"reg"))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x12-" + tag+"reg")
            except:
                print("Unable to build model for:", algos[j])
                traceback.print_exc()
        if tunemodel[j]:
            if algos[j] in ['dt', 'lightgbm']:
                try:
                    dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=15, verbose=True)
                    save_model(dtmodels[j], root+"/models/" + algos[j] + "-x12-"+ tag +"reg")
                except:
                    print("Unable to tune for: ", algos[j])
                    traceback.print_exc()

##stacked=stack_models(dtmodels, meta_model=dtmodels[len(dtmodels)-1])
##save_model(stacked, root+"/models/" + "x12-" + str(i) + "stackedreg")
#    try:
#        stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
#        save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    except:
#        print("Unable to tune for stacking")
##dtmodels.append(stacked)

x12-0dtcnt.csv x12-1dtcnt.csv x12-2dtcnt.csv x12-3dtcnt.csv Loaded data... train count: (80360, 2602) target
0    61621
1    18739
Name: customer_ID, dtype: int64


IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:40:10
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


In [ ]:
dtmodels[0] = tune_model(dtmodels[0], fold=10, search_library = "scikit-optimize", n_iter=50, verbose=True)


In [ ]:
traindf = pd.read_csv(root+"/xall.csv")
s = setup(data=traindf, target='target',
    transformation=True,
    normalize=True,
    ignore_features=['customer_ID'],
    categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
    numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
#            remove_outliers=True,
    handle_unknown_categorical=True,
#            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
#    trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=d,
    feature_selection=True, remove_multicollinearity=True, #pca=True,
    ignore_low_variance=True,
    train_size=0.9, fold=10,
    use_gpu=True,
    silent = True, session_id = 123, verbose=True)


In [ ]:
model = create_model('lightgbm', verbose=True)
evaluate_model(model)

In [ ]:
tuned = tune_model(model, search_library = "scikit-optimize", verbose=True)

In [ ]:
evaluate_model(tuned)

In [ ]:
blend = blend_models(dtmodels)

In [ ]:
finmodel = finalize_model(blend)

In [ ]:
save_model(model, root+"/models/lightgbm-xall")
plot_model(model, 'error')

In [ ]:
save_model(tuned, root+"/models/tuned-lightgbm-xall")
plot_model(tuned, 'error')

In [ ]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=18
algos=['dt', 'catboost', 'lightgbm']

for i in range(1):
#    for algo in algos:
#        if algo == "catboost":
#            dtmodels.append(load_model(root+"/models/" + algo + str(n)+"-"+str(i)+"reg"))
#    dtmodels.append(load_model(root+"/models/" + "".join(algos) + str(n) + "-" + str(i) + "stackedreg"))
    dtmodels.append(load_model(root+"/models/" + "x" + str(i) + "stackedreg"))

In [ ]:
stacked=stack_models(dtmodels, meta_model=dtmodels[0])
#stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
dtmodels.append(stacked)
save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")

In [ ]:
tests = []
for r, d, f in os.walk(root+"test"):
#    print(r)
#    if r ==root+'/test':
        f.sort()
        for file in f:
            if '.idx' not in file and '.csv' not in file and 'cust' not in file and 'preds' not in file and ('testa' in file or 'testb' in file or 'testc' in file or 'testd' in file or 'teste' in file or 'testf' in file):
                tests.append(os.path.join(r, file))
len(tests)#, tests[len(tests)-1]

In [ ]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

testdf = pd.read_csv(root+"/tstdata")
tdf = testdf.groupby('customer_ID')['S_2'].agg(['min', 'max', 'count']).reset_index()
for i in range(1, 14):
    print(i, tdf[tdf['count']==i]['count'].count())

tdf.rename(columns={"min": "mindate", "max": "maxdate"}, inplace=True)

tdf['maxdate'] = pd.to_datetime(tdf['maxdate'])
tdf['mindate'] = pd.to_datetime(tdf['mindate'])

#tdf['diffcount'] = tdf.apply(lambda x: ((x['maxdate'] - x['mindate'])/np.timedelta64(1, 'M')))

tdf['mondiff'] = (tdf.maxdate.dt.strftime("%Y%m").astype(int) - tdf.mindate.dt.strftime("%Y%m").astype(int) - 87)
tdf['mondiff'] = tdf['mondiff'].apply(lambda x: x+88 if x< 0 else x)
tdf[tdf['count'] != tdf['mondiff']]

In [ ]:
testdfs = []

for k in range(13):
    testdfs.append(pd.DataFrame(columns=headers))

k=0
for i in range(len(tests)):
    print(tests[i])
    testdf = pd.read_csv(tests[i], names=headers, header=None)
    for j in range(1, 14):
        mask = testdf.customer_ID.isin(tdf[tdf['mondiff'] == j].customer_ID)
        testdfs[j-1] = pd.concat([testdfs[j-1], testdf[mask]], ignore_index=True)
        if testdfs[j-1]['customer_ID'].count() >= 260000:
            testdfs[j-1].iloc[:260000,].to_csv("t" + str(j-1) + "-" + str(k) + ".csv", index=False)
            print("t" + str(i) + "-" + str(k) + ".csv")
            testdfs[j-1] = testdfs[j-1].iloc[260000:,]
            k+=1

for i in range(12):
    testdfs[i].to_csv("t" + str(i) + ".csv", index=False)
    print("t" + str(i) + ".csv")

for i in range(12, len(testdfs)):
    testdfs[i].to_csv("t" + str(i) + "-" + str(k) + ".csv", index=False)
    print("t" + str(i) + "-" + str(k) + ".csv")


In [ ]:
testdfs = []

for k in range(13):
    testdfs.append(pd.DataFrame(columns=headers))
k=0
for i in range(len(tests)):
    print(tests[i])
    testdf = pd.read_csv(tests[i], names=headers, header=None)
    for j in range(13, 14):
        mask = testdf.customer_ID.isin(tdf[tdf['S_2']==j].customer_ID)
        testdfs[j-1] = pd.concat([testdfs[j-1], testdf[mask]], ignore_index=True)
        if testdfs[j-1]['customer_ID'].count() >= 260000:
            testdfs[j-1].iloc[:260000,].to_csv("t" + str(j-1) + "-" + str(k) + ".csv", index=False)
            testdfs[j-1] = testdfs[j-1].iloc[260000:,]
            k+=1            

testdfs[12].to_csv("t12-" + str(k) + ".csv", index=False)


In [ ]:
predictions=pd.DataFrame()
for i in range(11, 12): #, 12):
    model = dtmodels[len(dtmodels)-1] #load_model(root+"/models/" + "x" + str(i) + "stackedreg")
    print("t" + str(i) + "dtcnt.csv")
    testdf = (pd.read_csv("t" + str(i) + "dtcnt.csv"))

    preds = predict_model(model, data=testdf, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
    preds[['customer_ID', 'Label']].to_csv(root+"/test/t"+ str(i) + ".dtcnt" + ".preds", header=False, index=False)

#predictions.to_csv("test.preds", header=False, index=False)

In [ ]:
testdfs = []
i=0
#mdf = pd.DataFrame()
#for m in range(2):
#    mdf = pd.concat([mdf, pd.read_csv("t12-"+str(m)+".csv")], ignore_index=True)
#mdf['tag'] = 12
#mdf['S_2'] = pd.to_datetime(mdf['S_2'])
for j in range(12):
    print("t" + str(j), end=" ")
    testdfs.append(pd.read_csv("t" + str(j) + ".csv"))

    maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
    mindate = maxdate + pd.offsets.DateOffset(months= -1-j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=dates['seq']+1

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonthorig'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i]['seq'] = testdfs[i].groupby(['customer_ID']).cumcount().add(1)
    testdfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = testdfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.strftime("%Y%m").astype(int) - (maxdate.year*100 + maxdate.month)

#    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2019 and x['mondiff'] > j else x['mondiff'], axis=1)
#    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <=0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")

#    testdfs[i]['ncount'] = testdfs[i].isnull().sum(axis=1)

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
#    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
#    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['yearmonthorig'].fillna(0, inplace=True)
#    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['mondiff'], inplace=True)
#    testdfs[i].drop(columns=['S_2'], inplace=True)

#    testdfs[i]['tag'] = 0
#    testdfs[i] = pd.concat([testdfs[i], mdf], ignore_index=True)

#    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
#    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
#    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
#    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['S_2'], inplace=True)

#    cols=testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#    testdfs[i].drop(testdfs[i][testdfs[i]['tag']==12].index, axis=0, inplace=True)
#    testdfs[i].drop(columns=['tag'], inplace=True)

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    testdfs[i].fillna(0, inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(-999, inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t" + str(j) +"13dts.csv", index=False)
    i+=1

testdfs[0]


In [ ]:
mdf=pd.read_csv("mdfdtcnt.csv")
for j in range(12):
    print(j, end=" ")
    pd.concat([mdf, traindfs[j]], ignore_index=True).to_csv("x" + str(j) + "13dts.csv", index=False)
    pd.concat([mdf[mdf.columns.tolist()[:len(mdf.columns)-2]], testdfs[j]], ignore_index=True).to_csv("t" + str(j) + "13dts.csv", index=False)


In [3]:
testdfs = []
i=0
maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
mindate = maxdate + pd.offsets.DateOffset(months= -13)
dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
dates.reset_index(inplace=True)
dates.rename(columns={'index': 'seq'}, inplace=True)
dates['seq']=dates['seq']+1

for j in range(12, 41):
    print("t12-" + str(j) + ".csv")
    testdfs.append(pd.read_csv("t12-" + str(j) + ".csv"))

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonthorig'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i]['seq'] = testdfs[i].groupby(['customer_ID']).cumcount().add(1)
    testdfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = testdfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.strftime("%Y%m").astype(int) - (maxdate.year*100 + maxdate.month)

#    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2019 and x['mondiff'] > j else x['mondiff'], axis=1)
#    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <=0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")

#    testdfs[i]['ncount'] = testdfs[i].isnull().sum(axis=1)

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
#    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
#    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['yearmonthorig'].fillna(0, inplace=True)
#    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['mondiff'], inplace=True)
#    testdfs[i].drop(columns=['S_2'], inplace=True)

#    testdfs[i]['tag'] = 0
#    testdfs[i] = pd.concat([testdfs[i], mdf], ignore_index=True)

#    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
#    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
#    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
#    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['S_2'], inplace=True)

#    cols=testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#    testdfs[i].drop(testdfs[i][testdfs[i]['tag']==12].index, axis=0, inplace=True)
#    testdfs[i].drop(columns=['tag'], inplace=True)

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    testdfs[i].fillna(0, inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(-999, inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t12-" + str(j) +"dtcnt.csv", index=False)
#    os.remove(tests[i]
    i+=1

testdfs[0]

t12-12.csv
t12-13.csv
t12-14.csv
t12-15.csv
t12-16.csv
t12-17.csv
t12-18.csv
t12-19.csv
t12-20.csv
t12-21.csv
t12-22.csv
t12-23.csv
t12-24.csv
t12-25.csv
t12-26.csv
t12-27.csv
t12-28.csv
t12-29.csv
t12-30.csv
t12-31.csv
t12-32.csv
t12-33.csv
t12-34.csv
t12-35.csv
t12-36.csv
t12-37.csv
t12-38.csv
t12-39.csv
t12-40.csv


,customer_ID,B_1201804,B_10201804,B_11201804,B_12201804,B_13201804,B_14201804,B_15201804,B_16201804,B_17201804,B_18201804,B_19201804,B_2201804,B_20201804,B_21201804,B_22201804,B_23201804,B_24201804,B_25201804,B_26201804,B_27201804,B_28201804,B_29201804,B_3201804,B_30201804,B_31201804,B_32201804,B_33201804,B_36201804,B_37201804,B_38201804,B_39201804,B_4201804,B_40201804,B_41201804,B_42201804,B_5201804,B_6201804,B_7201804,B_8201804,B_9201804,D_102201804,D_103201804,D_104201804,D_105201804,D_106201804,D_107201804,D_108201804,D_109201804,D_110201804,D_111201804,D_112201804,D_113201804,D_114201804,D_115201804,D_116201804,D_117201804,D_118201804,D_119201804,D_120201804,D_121201804,D_122201804,D_123201804,D_124201804,D_125201804,D_126201804,D_127201804,D_128201804,D_129201804,D_130201804,D_131201804,D_132201804,D_133201804,D_134201804,D_135201804,D_136201804,D_137201804,D_138201804,D_139201804,D_140201804,D_141201804,D_142201804,D_143201804,D_144201804,D_145201804,D_39201804,D_41201804,D_42201804,D_43201804,D_44201804,D_45201804,D_46201804,D_47201804,D_48201804,D_49201804,D_50201804,D_51201804,D_52201804,D_53201804,D_54201804,D_55201804,D_56201804,D_58201804,D_59201804,D_60201804,D_61201804,D_62201804,D_63201804,D_64201804,D_65201804,D_66201804,D_68201804,D_69201804,D_70201804,D_71201804,D_72201804,D_73201804,D_74201804,D_75201804,D_76201804,D_77201804,D_78201804,D_79201804,D_80201804,D_81201804,D_82201804,D_83201804,D_84201804,D_86201804,D_87201804,D_88201804,D_89201804,D_91201804,D_92201804,D_93201804,D_94201804,D_96201804,P_2201804,P_3201804,P_4201804,R_1201804,R_10201804,R_11201804,R_12201804,R_13201804,R_14201804,R_15201804,R_16201804,R_17201804,R_18201804,R_19201804,R_2201804,R_20201804,R_21201804,R_22201804,R_23201804,R_24201804,R_25201804,R_26201804,R_27201804,R_28201804,R_3201804,R_4201804,R_5201804,R_6201804,R_7201804,R_8201804,R_9201804,S_11201804,S_12201804,S_13201804,S_15201804,S_16201804,S_17201804,S_18201804,S_19201804,S_20201804,S_22201804,S_23201804,S_24201804,S_25201804,S_26201804,S_27201804,S_3201804,S_5201804,S_6201804,S_7201804,S_8201804,S_9201804,bncnt201804,day_of_month201804,day_of_week201804,day_of_year201804,dncnt201804,origmissrcnt201804,pncnt201804,rncnt201804,sncnt201804,totmissrcnt201804,week_of_year201804,yearmonthorig201804,B_1201805,B_10201805,B_11201805,B_12201805,B_13201805,B_14201805,B_15201805,B_16201805,B_17201805,B_18201805,B_19201805,B_2201805,B_20201805,B_21201805,B_22201805,B_23201805,B_24201805,B_25201805,B_26201805,B_27201805,B_28201805,B_29201805,B_3201805,B_30201805,B_31201805,B_32201805,B_33201805,B_36201805,B_37201805,B_38201805,B_39201805,B_4201805,B_40201805,B_41201805,B_42201805,B_5201805,B_6201805,B_7201805,B_8201805,B_9201805,D_102201805,D_103201805,D_104201805,D_105201805,D_106201805,D_107201805,D_108201805,D_109201805,D_110201805,...,R_21201903,R_22201903,R_23201903,R_24201903,R_25201903,R_26201903,R_27201903,R_28201903,R_3201903,R_4201903,R_5201903,R_6201903,R_7201903,R_8201903,R_9201903,S_11201903,S_12201903,S_13201903,S_15201903,S_16201903,S_17201903,S_18201903,S_19201903,S_20201903,S_22201903,S_23201903,S_24201903,S_25201903,S_26201903,S_27201903,S_3201903,S_5201903,S_6201903,S_7201903,S_8201903,S_9201903,bncnt201903,day_of_month201903,day_of_week201903,day_of_year201903,dncnt201903,origmissrcnt201903,pncnt201903,rncnt201903,sncnt201903,totmissrcnt201903,week_of_year201903,yearmonthorig201903,B_1201904,B_10201904,B_11201904,B_12201904,B_13201904,B_14201904,B_15201904,B_16201904,B_17201904,B_18201904,B_19201904,B_2201904,B_20201904,B_21201904,B_22201904,B_23201904,B_24201904,B_25201904,B_26201904,B_27201904,B_28201904,B_29201904,B_3201904,B_30201904,B_31201904,B_32201904,B_33201904,B_36201904,B_37201904,B_38201904,B_39201904,B_4201904,B_40201904,B_41201904,B_42201904,B_5201904,B_6201904,B_7201904,B_8201904,B_9201904,D_102201904,D_103201904,D_104201904,D_105201904,D_106201904,D_107201904,D_108201904,D_109201904,D_110201904,D_111201904,D_112201904,D_113201904,D_114201904,D_11520

In [ ]:
testdfs[0]
#dateshift.sort_values('mondiff', ascending=False)
#dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
#dateshift['min'] = pd.to_datetime(dateshift['min'])
#dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])

#dateshift['mondiff'] = dateshift['S_2'].dt.strftime("%Y%m").astype(int) - dateshift['min'].dt.strftime("%Y%m").astype(int) - 87
#dateshift[dateshift['customer_ID'] == '064b1f0072facb6599248737d0b1c9f011eba59233794057beb66b1dc13d647a']

testdfs[0][testdfs[i]['customer_ID'] == '064b1f0072facb6599248737d0b1c9f011eba59233794057beb66b1dc13d647a']

#pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()

testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
tst = testdfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1).reset_index()
#testdfs[i][testdfs[i]['customer_ID'] == '064aee43581981bdcf7e2906f8cb875a503445ed7ac2d40ad68a03c213398e79']
#tst[tst['customer_ID'] == '064aee43581981bdcf7e2906f8cb875a503445ed7ac2d40ad68a03c213398e79']

dateshift

dateshift['S_2'].dt.strftime("%Y%m").astype(int) - (maxdate.year*100 + maxdate.month) 

In [ ]:
#model = load_model("/amex/models/dt1Lclass")
#model = calibrated_dt
#model=stacked
#model=load_model("/amex/models/dtstackedreg")

useFlag = False
mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

predictions=pd.DataFrame()
for i in range(104, len(tests)):
    print(tests[i].split("/")[5], end =" ")
    if (not useFlag):
        headers=pd.read_csv(root+"headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i], names=headers, header=None)
        test['S_2'] = pd.to_datetime(test['S_2'])

        dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
        dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
        dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

        test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
        test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
        test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

        test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
        test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)

        test['day_of_month'] = test['S_2'].dt.strftime("%d")
        test['day_of_week'] = test['S_2'].dt.strftime("%u")
        test['day_of_year'] = test['S_2'].dt.strftime("%j")
        test.drop(columns=['S_2_dt'], inplace=True)
        test.drop(columns=['mondiff'], inplace=True)

        test = test.set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
        test = test.reset_index()
        test = pd.merge(test, dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

        test = test.merge(labels, on="customer_ID", how="left")
        test.drop(columns=[test.columns.to_list()[1]], inplace=True)

        coldict={}
        for col in test.columns:
            coldict[col] = ''.join(col)

        test.rename(coldict, axis=1, inplace=True)
        test['target'] = 0

#        test.to_csv(tests[i]+".csv", index=False, header=False)
    else:
        headers=pd.read_csv(root+"/trains/headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i]+".csv", names=headers, header=None)

    for col in [col for col in test.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
        test[col] = test[col].astype(str)
        test[col] = test[col].astype('category')

    test['customer_ID2'] = test['customer_ID']
    test.drop(columns=['customer_ID'], inplace=True)
    test.rename(columns = {"customer_ID2": "customer_ID"}, inplace=True)
    try:
        preds = predict_model(dtmodels[len(dtmodels)-1], data=test, verbose=True)
    except:
        preds = predict_model(dtmodels[0], data=test, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
#    preds[['customer_ID', 'Label']].to_csv(tests[i]+"." + "".join(algos) + s
    preds[['customer_ID', 'Label']].to_csv(tests[i]+".dt2" + ".preds", header=False, index=False)

predictions.to_csv("test.preds", header=False, index=False)

In [ ]:
from datetime import datetime

mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

test = pd.read_csv(tests[i], names=headers, header=None)
for col in [col for col in test.columns if col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]:
    test[col] = test[col].astype(str)
    test[col] = test[col].astype('category')

test['S_2'] = pd.to_datetime(test['S_2'])

dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].min()).reset_index()
dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)


In [ ]:
print((test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*12 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month) - (mindate.year*12+mindate.month), (test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*100 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month), (mindate.year*100+mindate.month))
test
test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']

In [ ]:
def testcolfillna(col):
#    print(col, end="")
    try:
        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]
    except:
        print(col)
    return

def testfillna(col):
#    print(".", end="")
    test[col].fillna(test[[column for column in test.columns.tolist() if column[:len(column)-3] in col]].mean(axis=1), inplace=True)

#    with Pool(12) as pool:
#        pool.map(testcolfillna, [col for col in test.columns[test.isna().all()]])

#    with Pool(12) as pool:
#        pool.map(testfillna, [col for col in test.columns[test.isna().any()]])

#    for col in test.columns[test.isna().all()]:
#        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]



In [ ]:
traindf = pd.read_csv(root+"/x12-0.csv")
traindf

In [ ]:
df=pd.read_csv(root+"/x3dtcnt.csv")
df[df['yearmonthorig201903'] == 0]